# Проект по SQL

Во время коронавируса наша компания  купила крупный сервис для чтения книг по подписке. Нам необходимо проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Цель исследования
- проанализировать базу данных крупного сервиса для чтения книг по подписке

### Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.



### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
#from sqlalchemy import create_engine, text as sql_text
#from sqlalchemy import text

# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})





### Исследование таблиц

Напишем функцию для вывода первых строк таблицы:

In [2]:
def request(query):
    con = engine.connect()
    result = pd.io.sql.read_sql(sql=text(query), con=con)
    return result

In [3]:
request('''SELECT *
           FROM books
           LIMIT 5
           ''')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
request('''SELECT *
           FROM authors
           LIMIT 5''')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
request('''SELECT *
           FROM publishers
           LIMIT 5''')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
request('''SELECT *
           FROM ratings
           LIMIT 5''')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
request('''SELECT *
           FROM reviews
           LIMIT 5''')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Количество книг, вышедших после 1 января 2000 года

In [8]:
request('''SELECT COUNT(book_id)
           FROM books
           WHERE publication_date  > '2000-01-01' ''')

,count
0,819


После 1 января  2000 года (не включая 01.01.2000) вышла 819 книг.

### Количество обзоров и средняя оценка для каждой книги

In [9]:
request('''SELECT b.title,
                  COUNT(DISTINCT review_id) AS count_review,
                  AVG(rating) AS average_rating
           FROM books AS b
           LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
           LEFT JOIN reviews AS rv ON rt.book_id = rv.book_id
           GROUP BY b.book_id
           ORDER BY 2 DESC''')

,title,count_review,average_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


Для каждой книги выведено количество обзоров, лидер здесь Twilight (Twilight #1)	- 7 обзоров, а также средняя оценка. В таблице нескольких книг со средним рейтингом - 5. A Dirty Job (Grim Reaper #1)	лидер по количеству обзоров среди книг с оценкой 5.

Для решения использована таблица books, ratings и reviews.


### Издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [10]:
request('''SELECT p.publisher,
                  COUNT(b.book_id) AS number_of_books
           FROM publishers AS p
           JOIN books AS b ON p.publisher_id = b.publisher_id
           WHERE b.num_pages > 50
           GROUP BY 1
           ORDER BY 2 DESC
           LIMIT 1''')

,publisher,number_of_books
0,Penguin Books,42


Наибольшее количество книг выпустило издание Penguin Books - 42 книги.

### Автор с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [11]:
request('''with t1 as (
                        SELECT b.book_id, 
                               b.title, 
                               COUNT(rating_id) AS count_rating, 
                               AVG(rating) AS average_rating
                        FROM books as b
                        JOIN ratings as r ON  b.book_id = r.book_id 
                        GROUP BY 1, 2
                                         )
          SELECT a.author_id, 
                 a.author, 
                 AVG(average_rating) 
          FROM authors as a
          JOIN books  ON books.author_id = a.author_id
          JOIN t1 ON t1.book_id = books.book_id
          WHERE count_rating >= 50
          GROUP BY 1, 2
          ORDER BY 3 DESC
          LIMIT 1''')

,author_id,author,avg
0,236,J.K. Rowling/Mary GrandPré,4.283844


Самая высокая средняя оценка книги среди книг с 50 и более оценками у J.K. Rowling/Mary GrandPré - 4.28. Серия книг о Harry Potter и правда потрясающая. Другие книги автора наверняка такие же интересные.

### Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
request('''
           SELECT COUNT(DISTINCT rv.text)/ COUNT(DISTINCT rv.username) AS avg_reviews
           FROM reviews AS rv
           JOIN ratings AS r ON rv.username = r.username
           WHERE r.username in (
                    SELECT username
                    FROM ratings
                    GROUP BY 1
                    HAVING COUNT(rating_id) > 48)
                                                  ''')

,avg_reviews
0,24


Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24 обзора. Оценку поставить проще, отзыв все-таки надо писать развернуто, не всегла есть время и желание.

### **Вывод:**

Мы проанализирвоали базу данных крупного сервиса для чтения книг по подписке:
- Общее количество книг в базе - 1000. С начала 2020 года коллекция книг пополнилась на 821 книгу. Книги разнообразны по авторам и издательствам. Это позволяет удовлетворить различные вкусы и предпочтения читателей.
- Много популярных авторов и книг с высоким рейтингом.
- Сервис предоставляет возможность оставлять обзоры и рейтинги, так точно можно найти книгу по душе, открыть для себя новых авторов.
- В базе много книг признанных издательств, выпускающих качественные книги и новинки.  
